In [18]:
 # A script that generates an adsorption corrections file when adsorbate and corresponding gas-phase thermodata.csv files as well as a .connectivity file for each adsorbate are present.
 # Example csv files, generated by copying data from the thermodata.py files, are included in this directory.
 # .connectivity files corresponding to the included csv files are in the C/H/N/O directories
 # This script is written by Katrín Blöndal at Brown University.

import sys, os
import fnmatch
import numpy as np
g=open("Pt_adsorption_corrections.py",'w')
g.write("#!/usr/bin/env python \n")
g.write("# encoding: utf-8 \n")
#Contains the thermochemistry corrections for adsorption on Pt(111) based on DFT frequency calculations of the adsorbed species and their gas-phase precursors.
g.write('\nname = "Surface Adsorption Corrections"\nshortDesc = u""\nlongDesc = u"""\nChanges due to adsorbing on a surface.\nHere, Pt(111)\nNote: "-h" means "horizontal"\n"""')
g.write('\n\nentry( \n    index = 1,')
g.write('\n    label = "R*",')
g.write('\n    group= \n"""\n1 R u0\n2 X u0\n""",\n')
g.write('    thermo=None,\n')
g.write('    shortDesc=u"""Anything adsorbed anyhow.""",\n')
g.write('    longDesc=u"""\n   R\n   x\n***********\nThis node should be empty, ensuring that one of the nodes below is used.\n""",\n)')
index = 1
path='./'
import csv
f = open('Pt_thermodata_adsorbates.py','r')
ads_reader = csv.reader(f)
input_csv_ads=[]
for row in ads_reader:
    input_csv_ads.append(row)
h = open('Pt_thermodata_gasphase.py','r')
gas_reader = csv.reader(h)
input_csv_gas=[]
for row in gas_reader:
    input_csv_gas.append(row)
ind = 0
enthalpy_list=[]
entropy_list=[]
cp_list=[]
for ads_row in input_csv_ads:
    path='R*single_chemisorbed/'    
    ads_name = ads_row[0].replace('_ads','').replace('[','')
    ind_gas = 0
    print(ads_name)
    for gas_row in input_csv_gas:
        gas_name = gas_row[0].replace('_ads','').replace('[','')
        if gas_name == ads_name:
            #print(gas_name)
            #print('match!')
            ind_gas +=1
            g.write('\n\nentry( \n    index = ' + str(index) + ',')
            g.write('\n    label = "' + str(ads_name.replace('H','R')) + '",')
            g.write('\n    group = \n"""\n')
            configfile = [os.path.join(dirpath, f)
                for dirpath, dirnames, files in os.walk(path)
                for f in fnmatch.filter(files, ads_name+'.connectivity')]
            #print(configfile)
            path = str(configfile[0])
            #print(path)
            input_file = open(path,'r')
            lines = input_file.readlines()
            input_file.close()
            line_count = 1
            for line in lines:
                g.write(line.replace('Pt','X ').replace('H','R'))
                if line_count == 1:
                    if line[-3] == 'S':
                        type_of_bond = 'single'
                    elif line[-3] == 'D':
                        type_of_bond = 'double'
                    elif line[-3] == 'T':
                        type_of_bond = 'triple'
                    elif line[-3] == 'Q':
                        type_of_bond = 'quadruple'
                    else:
                        type_of_bond = 'vdW'
                elif line_count == 2 and line[2] == 'P':
                    if line[-3] == 'S':
                        if type_of_bond == 'single':
                            type_of_bond = 'bidentate, twice single'
                        elif type_of_bond == 'double':
                            type_of_bond = 'bidentate, double- and single'
                        elif type_of_bond == 'triple':
                            type_of_bond = 'bidentate, triple- and single'
                    elif line[-3] == 'D':
                        if type_of_bond == 'single':
                            type_of_bond = 'bidentate, single- and double'
                        elif type_of_bond == 'double':
                            type_of_bond = 'bidentate, twice double'
                        elif type_of_bond == 'triple':
                            type_of_bond = 'bidentate, triple- and double'
                    elif line[-3] == 'T':
                        if type_of_bond == 'single':
                            type_of_bond = 'bidentate, single- and triple'
                        elif type_of_bond == 'double':
                            type_of_bond = 'bidentate, double- and triple'
                        elif type_of_bond == 'triple':
                            type_of_bond = 'bidentate, twice triple'
                    else:
                        print("Something is wrong")        
        
                line_count +=1
            g.write('""",\n')
            g.write("    thermo=ThermoData(\n        Tdata=([300, 400, 500, 600, 800, 1000, 1500], 'K'),\n")
            Cpdiff=[]
            for i in range(2,9):
                Cpdiff.append(round(float(ads_row[i])-float(gas_row[i]),2))
            g.write('        Cpdata=(' + str(Cpdiff) + ", 'J/(mol*K)'),\n")
            g.write('        H298=(' + str(round(float(ads_row[12])-float(gas_row[12]),2)) + ", 'kJ/mol'),\n")
            g.write('        S298=(' + str(round(float(ads_row[15])-float(gas_row[15]),2)) + ", 'J/(mol*K)'),\n    ),\n")
            g.write('    shortDesc=u"""Came from ' + str(ads_name) + ' ' + str(type_of_bond) + '-bonded on Pt(111)""",\n')
            g.write('    longDesc=u"""Calculated by Bjarne Kreitz at Brown University using statistical mechanics (files: compute_NASA_for_Pt-adsorbates.ipynb and compute_NASA_for_Pt-gas_phase.ipynb).\n')
            g.write("            Based on DFT calculations by Bjarne Kreitz from Brown University. DFT calculations were performed with Quantum Espresso\n")
            g.write("            using PAW pseudopotentials and the BEEF-vdW functional for an optimized 3x3 supercell (1/9ML coverage)\n")
            g.write("            following the procedure outlined by Blondal et al (DOI:10.1021/acs.iecr.9b01464). The following settings were applied:\n")    
            g.write("            kpoints=(5x5x1), 4 layers (2 bottom layers fixed), ecutwfc=60 Ry, smearing='mazari-vanderbilt', mixing_mode='local-TF',fmax=2.5e-2.\n***********\n")
            g.write('""",\n') 
            g.write('    metal = "Pt",\n')
            g.write('    facet = "111",\n)')
          
            index +=1
            
            cp_list.append(Cpdiff)
            new_list=np.array(cp_list)
            enthalpy_list.append(float(ads_row[12])-float(gas_row[12]))
            entropy_list.append(float(ads_row[15])-float(gas_row[15]))
            break
        else:
            ind_gas +=1
    ind +=1

average_cp=[]
for i in range(len(new_list[0,:])):
    average_cp.append(np.round(np.mean(new_list[:,i]),2))
average_enthalpy=np.round(np.mean(enthalpy_list),2)
average_entropy=np.round(np.mean(entropy_list),2)


g.write('\n\n\n\n')
g.write("    thermo=ThermoData(\n        Tdata=([300, 400, 500, 600, 800, 1000, 1500], 'K'),\n")
g.write('        Cpdata=(' + str(average_cp) + ", 'J/(mol*K)'),\n")
g.write('        H298=(' + str(average_enthalpy) + ", 'kJ/mol'),\n")
g.write('        S298=(' + str(average_entropy) + ", 'J/(mol*K)'),\n    ),\n")
g.write('    shortDesc=u"""Came from averaged on Pt(111)""",\n')
g.write('    longDesc=u"""Calculated by Bjarne Kreitz at Brown University using statistical mechanics (files: compute_NASA_for_Pt-adsorbates.ipynb and compute_NASA_for_Pt-gas_phase.ipynb).\n')
g.write("            Based on DFT calculations by Bjarne Kreitz from Brown University. DFT calculations were performed with Quantum Espresso\n")
g.write("            using PAW pseudopotentials and the BEEF-vdW functional for an optimized 3x3 supercell (1/9ML coverage)\n")
g.write("            following the procedure outlined by Blondal et al (DOI:10.1021/acs.iecr.9b01464). The following settings were applied:\n")    
g.write("            kpoints=(5x5x1), 4 layers (2 bottom layers fixed), ecutwfc=60 Ry, smearing='mazari-vanderbilt', mixing_mode='local-TF',fmax=2.5e-2.\n***********\n")
g.write('""",\n') 
g.write('    metal = "Pt",\n')
g.write('    facet = "111",\n)')

g.close()
print(average_cp)
print(average_enthalpy)
print(average_entropy)

CH2CH2CH3
CH2CH2OH
CH2CH3
CH2CHCH2-mono
CH2CHO-mono
CH3
CH3CHCH3
CH3CHOH
H2COH
CH2CCH3
CH2COH
CH3CO
CHCCH2
CHCH2
CHCHCH3-mono
CHCO-mono
COOH
HCO-mono
C
CH
COH
CCHCH2
CCH3
CCH2CH3
CH2
CH3CCH3
CH3COH
CHCH2CH3
CHCH3
CHCHCH2-mono
CHCHO-mono
HCOH
CCO
CCH2-mono
CCCH2-mono
OH
HCO3
HCOO-mono
OCHCH2
OCH3
OCH2CH3
OOH
O
CH3CH2CO
CCHO
CCH2OH
OCH2OH
[0.84, 4.02, 6.08, 7.46, 9.09, 9.97, 10.96]
-331.96
-169.67
